# 3. Extração de eventos de tramitação no Congresso



## 3.1. Preparação



## 3.1.1. Imports

In [1]:
import os

from pathlib import Path

import duckdb
import igraph as ig
import pandas as pd

from event import Event  # Enum customizado nosso para eventos relevantes de tramitação

from dotenv import load_dotenv

load_dotenv()

PROJECT_DIR = Path("~/tramita").expanduser()
DB_PATH = PROJECT_DIR / os.getenv("SILVER_DUCKDB_PATH", "")
OUT_DIR = PROJECT_DIR / "gold" / "outputs"
OUT_DIR.mkdir(exist_ok=True)
ACCESS_DIR = PROJECT_DIR / "gold" / "accessory_data"
ACCESS_DIR.mkdir(exist_ok=True)

NODES_PATH_PARQUET = OUT_DIR / "nodes.parquet"
EDGES_PATH_PARQUET = OUT_DIR / "edges.parquet"
NODES_PATH_CSV = OUT_DIR / "nodes.csv"
EDGES_PATH_CSV = OUT_DIR / "edges.csv"

## 3.1.2. Leitura do banco de dados

In [2]:
with duckdb.connect(DB_PATH, read_only=True) as con:
    house_props_df = con.execute("SELECT * FROM proposicoes_camara").df().set_index('id_proposicao', drop=True)
    house_tram_df = con.execute("SELECT * FROM tramitacoes_camara").df()
    house_autores_df = con.execute("SELECT * FROM autores_camara").df().set_index('id_autor', drop=True)
    house_deputados_df = con.execute("SELECT * FROM deputados_camara").df().set_index('id_deputado', drop=True)
    house_orgaos_df = con.execute("SELECT * FROM orgaos_camara").df().set_index('id_orgao', drop=True)
    senate_procs_df = con.execute("SELECT * FROM processo_senado").df().set_index('id_processo', drop=True)
house_tram_df = house_tram_df[house_tram_df['id_proposicao'].isin(house_props_df.index)].copy().set_index('id_tramitacao', drop=True)
house_tram_df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,id_proposicao,ambito,apreciacao,cod_situacao,cod_tipo_tramitacao,data_hora,descricao_situacao,descricao_tramitacao,despacho,regime,sequencia,sigla_orgao,uri_orgao,uri_ultimo_relator,year_snapshot
id_tramitacao,,,,,,,,,,,,,,,
462,538196,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,100,2021-11-17 00:00:00,Transformado em Norma Jurídica,Apresentação de Proposição,"Ofício nº 205/2021/PS-GSE ao Senado Federal, i...","Urgência (Art. 155, RICD)",213,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020
545,559138,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,100,2025-05-05 00:00:00,Transformado em Norma Jurídica,Apresentação de Proposição,"Apresentação do Of. nº 10 /2024/PS-GSE, que co...","Urgência (Art. 155, RICD)",262,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020
647,593065,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,100,2021-11-17 00:00:00,Transformado em Norma Jurídica,Apresentação de Proposição,"Ofício nº 201/2021/PS-GSE ao Senado Federal, ...","Urgência (Art. 155, RICD)",322,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020
687,601739,Regimental,Proposição Sujeita à Apreciação do Plenário,924,501,2024-12-02 18:35:00,Pronta para Pauta,Recebimento - Relator(a),Apresentação do PRL n. 1 CAPADR (Parecer do Re...,"Ordinário (Art. 151, III, RICD)",136,CAPADR,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020
744,614512,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,500,2022-07-15 00:00:00,Transformado em Norma Jurídica,Recebimento,Recebimento do Ofício nº 306/2022 (CN) encamin...,"Urgência (Art. 155, RICD)",203,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880670,2466104,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,194,2024-11-04 14:35:00,Transformado em Norma Jurídica,Apresentação de Requerimento,Apresentação do REQ n. 4403/2024 (Requerimento...,"Urgência (Art. 155, RICD)",13,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2024
880671,2466104,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,110,2024-11-04 13:13:00,Transformado em Norma Jurídica,Distribuição,Às Comissões de Administração e Serviço Públic...,"Urgência (Art. 155, RICD)",8,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2024
880672,2466104,Regimental,Proposição Sujeita à Apreciação do Plenário,None,604,2024-11-04 00:00:00,None,Publicação de Proposição,Encaminhada à publicação. Publicação Inicial e...,"Urgência (Art. 155, RICD)",14,CCP,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2024


## 3.2. Construção da tabela de tramitações

A partir daqui vamos executar uma série de passos para cosntruir uma tabela completa com as tramitações de cada proposição em cada casa

### 3.2.1. Câmara dos Deputados

In [3]:
house_tram_expanded_df = house_tram_df.join(house_props_df, on="id_proposicao", how="left", lsuffix="tram", rsuffix="props")[[
    'id_proposicao',
    'prop_label',
    'sequencia',
    'data_hora',
    'descricao_tramitacao',
    'sigla_orgao',
    'despacho',
    'cod_situacao',
    'descricao_situacao',
    'regime',
    'apreciacao',
    'cod_tipo_tramitacao',
    'uri_orgao',
    # 'prop_tag',
]].sort_values(['id_proposicao', 'data_hora'])
house_tram_expanded_df

,id_proposicao,prop_label,sequencia,data_hora,descricao_tramitacao,sigla_orgao,despacho,cod_situacao,descricao_situacao,regime,apreciacao,cod_tipo_tramitacao,uri_orgao
id_tramitacao,,,,,,,,,,,,,
771001,253500,PL 2597/2024,1,2004-05-13 14:41:00,Apresentação de Proposição,PLEN,Apresentação do Projeto de Lei pelo Dep. José ...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,100,https://dadosabertos.camara.leg.br/api/v2/orga...
771000,253500,PL 2597/2024,6,2004-05-14 10:46:00,Recebimento,SECAP(SGM),Recebimento pela MESA.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...
770999,253500,PL 2597/2024,12,2004-05-28 11:08:00,Distribuição,MESA,"Às Comissões de Desenvolvimento Econômico, Ind...",1140,Transformado em Norma Jurídica,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,110,https://dadosabertos.camara.leg.br/api/v2/orga...
770998,253500,PL 2597/2024,17,2004-06-01 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial n...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,604,https://dadosabertos.camara.leg.br/api/v2/orga...
770997,253500,PL 2597/2024,18,2004-06-02 09:45:00,Recebimento,CDE,Recebimento pela CDEIC.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
875892,2531284,PL 3194/2025,2,2025-07-02 14:10:00,Apresentação de Proposição,MESA,Apresentação do PL n. 3194/2025 (Projeto de Le...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,100,https://dadosabertos.camara.leg.br/api/v2/orga...
875352,2531284,PL 3194/2025,8,2025-08-29 14:45:00,Despacho de Apensação,MESA,Apense-se à(ao) PL-4348/2024.Proposição Sujeit...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,129,https://dadosabertos.camara.leg.br/api/v2/orga...
874734,2531284,PL 3194/2025,14,2025-09-02 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial e...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,604,https://dadosabertos.camara.leg.br/api/v2/orga...


Não tínhamos todos os dados dos órgãos da Câmara, então baixamos da API.

In [4]:
# import asyncio
# import httpx
# orgaos_df = house_tram_expanded_df[['sigla_orgao', 'uri_orgao']].drop_duplicates()
# async def get_data(uri: str, client: httpx.AsyncClient, sem: asyncio.Semaphore):
#     async with sem:
#         print(uri)
#         response = await client.get(uri)
#         response.raise_for_status()
#         return response.json()

# sem = asyncio.Semaphore(4)
# async with httpx.AsyncClient() as client:
#     tasks = [get_data(uri, client, sem) for uri in orgaos_df['uri_orgao']]
#     result = await asyncio.gather(*tasks)
# rows = [r['dados'] for r in result]
# full_orgaos_df = pd.DataFrame(rows).set_index('uri', drop=True)
# full_orgaos_df.to_pickle(ACCESS_DIR / 'full_orgaos_camara_df.pkl')


In [5]:
full_orgaos_df = pd.read_pickle(ACCESS_DIR / 'full_orgaos_camara_df.pkl')

In [6]:
house_tram_expanded_df = house_tram_expanded_df.join(
    full_orgaos_df[['nome', 'codTipoOrgao', 'tipoOrgao']],
    on="uri_orgao",
    how="left",
).rename(columns={'codTipoOrgao': 'cod_tipo_orgao', 'tipoOrgao': 'tipo_orgao', 'nome': 'nome_orgao'}).copy()
house_tram_expanded_df['data_hora'] = pd.to_datetime(house_tram_expanded_df['data_hora'])
house_tram_expanded_df

,id_proposicao,prop_label,sequencia,data_hora,descricao_tramitacao,sigla_orgao,despacho,cod_situacao,descricao_situacao,regime,apreciacao,cod_tipo_tramitacao,uri_orgao,nome_orgao,cod_tipo_orgao,tipo_orgao
id_tramitacao,,,,,,,,,,,,,,,,
771001,253500,PL 2597/2024,1,2004-05-13 14:41:00,Apresentação de Proposição,PLEN,Apresentação do Projeto de Lei pelo Dep. José ...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,100,https://dadosabertos.camara.leg.br/api/v2/orga...,Plenário,26,Plenário Virtual
771000,253500,PL 2597/2024,6,2004-05-14 10:46:00,Recebimento,SECAP(SGM),Recebimento pela MESA.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,Seção de Registro e Controle de Análise da Pro...,12000,Órgão da Câmara dos Deputados
770999,253500,PL 2597/2024,12,2004-05-28 11:08:00,Distribuição,MESA,"Às Comissões de Desenvolvimento Econômico, Ind...",1140,Transformado em Norma Jurídica,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,110,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora
770998,253500,PL 2597/2024,17,2004-06-01 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial n...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,604,https://dadosabertos.camara.leg.br/api/v2/orga...,COORDENAÇÃO DE COMISSÕES PERMANENTES,12000,Órgão da Câmara dos Deputados
770997,253500,PL 2597/2024,18,2004-06-02 09:45:00,Recebimento,CDE,Recebimento pela CDEIC.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Desenvolvimento Econômico,2,Comissão Permanente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875892,2531284,PL 3194/2025,2,2025-07-02 14:10:00,Apresentação de Proposição,MESA,Apresentação do PL n. 3194/2025 (Projeto de Le...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,100,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora
875352,2531284,PL 3194/2025,8,2025-08-29 14:45:00,Despacho de Apensação,MESA,Apense-se à(ao) PL-4348/2024.Proposição Sujeit...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,129,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora
874734,2531284,PL 3194/2025,14,2025-09-02 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial e...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,604,https://dadosabertos.camara.leg.br/api/v2/orga...,COORDENAÇÃO DE COMISSÕES PERMANENTES,12000,Órgão da Câmara dos Deputados


Excluímos alguns órgãos que não são relevantes ou corretos.

In [7]:
orgao_code_excludes = [
    70_000,  # Sociedade Civil (provavelmente erro)
    12_000,  # Órgãos burocráticos da Câmara (só protocolar)
    40_000,  # Senado Federal; veremos lá e não cá
    10,      # Grupos de trabalho, nada relevante
    11,      # Conselho de Ética da Câmara; irrelevante
]

house_tram_expanded_filtered_df = house_tram_expanded_df[
    (~house_tram_expanded_df['cod_tipo_orgao'].isin(orgao_code_excludes))
].sort_values(['id_proposicao', 'sequencia'])
house_tram_expanded_filtered_df

,id_proposicao,prop_label,sequencia,data_hora,descricao_tramitacao,sigla_orgao,despacho,cod_situacao,descricao_situacao,regime,apreciacao,cod_tipo_tramitacao,uri_orgao,nome_orgao,cod_tipo_orgao,tipo_orgao
id_tramitacao,,,,,,,,,,,,,,,,
771001,253500,PL 2597/2024,1,2004-05-13 14:41:00,Apresentação de Proposição,PLEN,Apresentação do Projeto de Lei pelo Dep. José ...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,100,https://dadosabertos.camara.leg.br/api/v2/orga...,Plenário,26,Plenário Virtual
770999,253500,PL 2597/2024,12,2004-05-28 11:08:00,Distribuição,MESA,"Às Comissões de Desenvolvimento Econômico, Ind...",1140,Transformado em Norma Jurídica,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,110,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora
770997,253500,PL 2597/2024,18,2004-06-02 09:45:00,Recebimento,CDE,Recebimento pela CDEIC.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Desenvolvimento Econômico,2,Comissão Permanente
770996,253500,PL 2597/2024,19,2004-06-08 11:34:00,Designação de Relator(a),CDE,"Designado Relator, Dep. Ronaldo Dimas (PSDB-TO)",None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,320,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Desenvolvimento Econômico,2,Comissão Permanente
770995,253500,PL 2597/2024,20,2004-06-08 11:37:00,Abertura de Prazo,CDE,Abertura de Prazo para Emendas ao Projeto a pa...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,350,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Desenvolvimento Econômico,2,Comissão Permanente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876366,2531284,PL 3194/2025,1,2024-04-15 00:00:00,Notificacao para Publicação Intermediária,CDC,"Designado Relator, Dep. Márcio Marinho (REPUBL...",925,Tramitando em Conjunto,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,1050,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Defesa do Consumidor,2,Comissão Permanente
875892,2531284,PL 3194/2025,2,2025-07-02 14:10:00,Apresentação de Proposição,MESA,Apresentação do PL n. 3194/2025 (Projeto de Le...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,100,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora
875352,2531284,PL 3194/2025,8,2025-08-29 14:45:00,Despacho de Apensação,MESA,Apense-se à(ao) PL-4348/2024.Proposição Sujeit...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,129,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora


A função abaixo extrai um `Event`, onde possível, para cada fileira da tabela de tramitações da Câmara

In [8]:
def house_row_to_event(row: pd.Series) -> Event | None:
    # apresentado
    if row['cod_tipo_tramitacao'] == '100':
        return Event.APRESENTADO

    # distribuído
    if row['cod_tipo_tramitacao'] == '110':
        return Event.DISTRIBUIDO

    # recebido em comissão
    if row['cod_tipo_tramitacao'] == '500' and row['tipo_orgao'].startswith('Comissão'):
        return Event.RECEBIDO_COMISSAO

    # designado relator em comissão
    if row['cod_tipo_tramitacao'] == '320' and row['tipo_orgao'].startswith('Comissão'):
        return Event.DESIGNADO_RELATOR_COMISSAO

    # retirado de pauta em comissão
    if row['cod_tipo_tramitacao'] == '250' and row['tipo_orgao'].startswith('Comissão'):
        return Event.RETIRADO_PAUTA_COMISSAO

    # aprovada urgência
    if row['cod_tipo_tramitacao'] == '196':
        return Event.APROVADA_URGENCIA

    # designado relator em plenário
    if row['cod_tipo_tramitacao'] == '320' and row['sigla_orgao'] == 'PLEN':
        return Event.DESIGNADO_RELATOR_PLENARIO

    # remetido
    if row['cod_tipo_tramitacao'] == '128':
        return Event.REMETIDO_AO_SENADO
    if row['cod_tipo_tramitacao'] == '609':
        return Event.REMETIDO_A_SANCAO
    if row['cod_tipo_tramitacao'] == '608':
        return Event.REMETIDO_A_PROMULGACAO
    if row['cod_tipo_tramitacao'] == '100' and row['despacho'].startswith('Remessa ao Senado Federal'):
        return Event.REMETIDO_AO_SENADO
    if row['cod_tipo_tramitacao'] == '1243' and row['despacho'].startswith('A matéria vai à sanção'):
        return Event.REMETIDO_A_SANCAO

    # aprovado em plenário
    if row['cod_tipo_tramitacao'] == '1235':
        return Event.APROVADO_PLENARIO

    # rejeitado em plenário
    # Não tem como saber ao certo
    pass

    # arquivado
    if row['cod_tipo_tramitacao'] in ['502', '1024']:
        return Event.ARQUIVADO

    # desarquivado
    if row['cod_tipo_tramitacao'] == '503':
        return Event.DESARQUIVADO

Consolidamos a tabela de eventos para a Câmara

In [9]:
df_with_events = house_tram_expanded_filtered_df.copy()
df_with_events['event'] = df_with_events.apply(house_row_to_event, axis=1)
df_with_events = df_with_events[df_with_events['event'].notnull()]
df_with_events

,id_proposicao,prop_label,sequencia,data_hora,descricao_tramitacao,sigla_orgao,despacho,cod_situacao,descricao_situacao,regime,apreciacao,cod_tipo_tramitacao,uri_orgao,nome_orgao,cod_tipo_orgao,tipo_orgao,event
id_tramitacao,,,,,,,,,,,,,,,,,
771001,253500,PL 2597/2024,1,2004-05-13 14:41:00,Apresentação de Proposição,PLEN,Apresentação do Projeto de Lei pelo Dep. José ...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,100,https://dadosabertos.camara.leg.br/api/v2/orga...,Plenário,26,Plenário Virtual,Event.APRESENTADO
770999,253500,PL 2597/2024,12,2004-05-28 11:08:00,Distribuição,MESA,"Às Comissões de Desenvolvimento Econômico, Ind...",1140,Transformado em Norma Jurídica,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,110,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,Event.DISTRIBUIDO
770997,253500,PL 2597/2024,18,2004-06-02 09:45:00,Recebimento,CDE,Recebimento pela CDEIC.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,Event.RECEBIDO_COMISSAO
770996,253500,PL 2597/2024,19,2004-06-08 11:34:00,Designação de Relator(a),CDE,"Designado Relator, Dep. Ronaldo Dimas (PSDB-TO)",None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,320,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,Event.DESIGNADO_RELATOR_COMISSAO
770990,253500,PL 2597/2024,28,2004-11-30 00:00:00,Distribuição,MESA,"Às Comissões de Desenvolvimento Econômico, Ind...",1140,Transformado em Norma Jurídica,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,110,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,Event.DISTRIBUIDO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756876,2529128,PL 3114/2025,14,2025-09-03 12:18:00,Recebimento,CSAUDE,Recebimento pelo(a) CSAUDE.,925,Tramitando em Conjunto,"Prioridade (Art. 151, II, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Saúde,2,Comissão Permanente,Event.RECEBIDO_COMISSAO
875891,2529248,PL 3127/2025,2,2025-06-30 15:09:00,Apresentação de Proposição,MESA,Apresentação do PL n. 3127/2025 (Projeto de Le...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,100,https://dadosabertos.camara.leg.br/api/v2/orga...,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,Event.APRESENTADO
874103,2529248,PL 3127/2025,14,2025-09-03 15:47:00,Recebimento,CMULHER,Recebimento pela CMULHER.,925,Tramitando em Conjunto,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,500,https://dadosabertos.camara.leg.br/api/v2/orga...,Comissão de Defesa dos Direitos da Mulher,2,Comissão Permanente,Event.RECEBIDO_COMISSAO


In [10]:

house_event_df = (df_with_events[['id_proposicao', 'prop_label', 'data_hora', 'event', 'sigla_orgao']]
    .sort_values(['data_hora', 'id_proposicao'])
    .reset_index(drop=True)
    .rename(columns={
        'data_hora': 'event_ts',
        'sigla_orgao': 'event_loc',
    }))


### 3.2.2. Senado Federal

In [11]:
with duckdb.connect(DB_PATH, read_only=True) as con:
    senate_desp_df = con.execute("SELECT * FROM despachos_senado").df().set_index('id_despacho', drop=True)
    senate_prov_df = con.execute("SELECT * FROM providencias_senado").df().set_index('id_providencia', drop=True)
    senate_sit_df = con.execute("SELECT * FROM situacoes_senado").df().set_index('id_situacao', drop=True)
    senate_aut_df = con.execute("SELECT * FROM autuacoes_senado").df()
    senate_inf_df = con.execute("SELECT * FROM informes_legislativos_senado").df().set_index('id_informe_legislativo', drop=True)
    senate_unid_df = con.execute("SELECT * FROM unidades_destinatarias_senado").df().set_index('id_unidade_destinataria', drop=True)
    
def filter_by_processo(df):
    return df[df['id_processo'].isin(senate_procs_df.index)]

senate_desp_df = filter_by_processo(senate_desp_df)
senate_prov_df = filter_by_processo(senate_prov_df)
senate_sit_df = filter_by_processo(senate_sit_df)
senate_aut_df = filter_by_processo(senate_aut_df)
senate_inf_df = filter_by_processo(senate_inf_df)
senate_unid_df = filter_by_processo(senate_unid_df)

In [12]:
senate_event_rows = []
for id_processo, row in senate_procs_df.iterrows():
    if not pd.isna(row['documento_data_apresentacao']):
        senate_event_rows.append(
            {
                'id_processo': id_processo,
                'proc_label': row['identificacao'],
                'event_ts': row['documento_data_apresentacao'],
                'event': Event.APRESENTADO,
                'event_loc': "SF",
            }
        )
        

Extraímos dos Informes Legislativos as designações de relator em comissão

In [13]:
senate_dist_df_ = senate_inf_df[senate_inf_df['descricao'].str.match(r"^(re)?distribuído", case=False)]
senate_dist_df_ = senate_dist_df_[senate_dist_df_['id_processo'].isin(senate_procs_df.index)]

for index, row in senate_dist_df_.iterrows():
    senate_event_rows.append({
        'id_processo': row['id_processo'],
        'proc_label': senate_procs_df.loc[row['id_processo'], 'identificacao'],
        'event_ts': row['data_informe'],
        'event': Event.DESIGNADO_RELATOR_COMISSAO,
        'event_loc': row['colegiado_sigla'],
    })

E das unidades de destino, a recepção em Comissão

In [14]:
senate_unid_ext_df = senate_unid_df.join(senate_desp_df, on="id_despacho", lsuffix="_unid", rsuffix="_desp")\
    .join(senate_prov_df, on="id_providencia", lsuffix="_unid", rsuffix="_prov")\
        .join(senate_procs_df[['identificacao']], on="id_processo")

# distribuído / recebido em comissão (dá na mesma aqui)
for index, row in senate_unid_ext_df.iterrows():
    senate_event_rows.append({
        'id_processo': row['id_processo_unid'],
        'proc_label': row['identificacao'],
        'event_ts': row['data_despacho'],
        'event': Event.RECEBIDO_COMISSAO,
        'event_loc': row['colegiado_sigla'],
    })

Da mesma forma, extraímos as aprovações de urgência, retiradas de pauta, designações de relator em plenário

In [15]:
def aprov_urgencia(descricao):
    for line in descricao.splitlines():
        line = line.lower()
        if "aprovado" in line and "urgência" in line:
            if line.index("aprovado") < line.index("urgência"):
                return True
    return False

In [16]:
# aprovada urgência
for index, row in senate_inf_df[senate_inf_df['descricao'].apply(aprov_urgencia)].iterrows():
    id_proc = row['id_processo']
    proc_label = senate_procs_df.loc[id_proc, 'identificacao']
    senate_event_rows.append({
        'id_processo': id_proc,
        'proc_label': proc_label,
        'event_ts': row['data_informe'],
        'event': Event.APROVADA_URGENCIA,
        'event_loc': row['colegiado_sigla'],
    })

In [17]:
def retirada_pauta(descricao):
    for line in descricao.splitlines():
        line = line.lower()
        if "a matéria foi retirada de pauta" in line or "a matéria é retirada de pauta" in line:
            return True
    return False

In [18]:
# retirada de pauta em comissão
for index, row in senate_inf_df[senate_inf_df['descricao'].apply(retirada_pauta)].iterrows():
    if row['colegiado_sigla'] == "PLEN":
        continue
    id_proc = row['id_processo']
    proc_label = senate_procs_df.loc[id_proc, 'identificacao']
    senate_event_rows.append({
        'id_processo': id_proc,
        'proc_label': proc_label,
        'event_ts': row['data_informe'],
        'event': Event.RETIRADO_PAUTA_COMISSAO,
        'event_loc': row['colegiado_sigla'],
    })

In [19]:
def relator_plenario(row):
    if row['colegiado_sigla'] != "PLEN":
        return False
    desc = row['descricao'].lower()
    return desc.startswith("designad") and "relator" in desc

In [20]:
for _, row in senate_inf_df[senate_inf_df.apply(relator_plenario, axis=1)].iterrows():
    id_proc = row['id_processo']
    proc_label = senate_procs_df.loc[id_proc, 'identificacao']
    senate_event_rows.append({
        'id_processo': id_proc,
        'proc_label': proc_label,
        'event_ts': row['data_informe'],
        'event': Event.DESIGNADO_RELATOR_PLENARIO,
        'event_loc': row['colegiado_sigla'],
    })

E finalmente as situações iniciadas conforme indicado nos informes legislativos

In [21]:
# demais situaçòes iniciadas
for _, row in senate_inf_df.iterrows():
    id_proc = row['id_processo']
    proc_label = senate_procs_df.loc[id_proc, 'identificacao']
    event_type = None
    sig = row['sigla_situacao_iniciada']
    if sig is None:
        continue
    if sig.startswith("APRVD"):
        event_type = Event.APROVADO_PLENARIO
    if sig == "ARQVD":
        event_type = Event.ARQUIVADO
    if sig == "RTPA":
        event_type = Event.RETIRADO_PAUTA_COMISSAO
    if sig == "RMSAN":
        event_type = Event.REMETIDO_A_SANCAO
    if sig == "RMCD":
        event_type = Event.REMETIDO_A_CAMARA
    if sig == "RMPRO":
        event_type = Event.REMETIDO_A_PROMULGACAO
    if sig.startswith("RJTDA"):
        event_type = Event.REJEITADO_PLENARIO
        
    if event_type is not None:
        senate_event_rows.append({
            'id_processo': id_proc,
            'proc_label': proc_label,
            'event_ts': row['data_informe'],
            'event': event_type,
            'event_loc': "PLEN",  # sometimes is None but all of this is plenário-related
        })

Construímos a tabela de eventos do Senado

In [22]:
senate_event_df = pd.DataFrame(senate_event_rows).sort_values(['event_ts', 'id_processo']).reset_index(drop=True)
senate_event_df

,id_processo,proc_label,event_ts,event,event_loc
0,7711601,PL 547/2019,2019-01-04,Event.APRESENTADO,SF
1,7717273,PL 635/2019,2019-02-01,Event.APRESENTADO,SF
2,7714029,PL 577/2019,2019-02-05,Event.APRESENTADO,SF
3,7714041,PL 581/2019,2019-02-05,Event.APRESENTADO,SF
4,7714045,PLP 16/2019,2019-02-05,Event.APRESENTADO,SF
...,...,...,...,...,...
138873,8776824,PL 3337/2024,2024-12-26,Event.REMETIDO_A_SANCAO,PLEN
138874,8779434,PL 3817/2024,2024-12-26,Event.REMETIDO_A_SANCAO,PLEN
138875,8781192,PL 4614/2024,2024-12-26,Event.REMETIDO_A_SANCAO,PLEN
138876,8781234,PL 3802/2024,2024-12-26,Event.REMETIDO_A_SANCAO,PLEN


### 3.2.3. Tabela conjunta

In [23]:
senate_event_df['casa'] = "senado"
house_event_df['casa'] = "camara"

In [24]:
full_event_df = pd.concat([
    house_event_df.drop('id_proposicao', axis=1).rename(columns={'prop_label': 'label'}),
    senate_event_df.drop('id_processo', axis=1).rename(columns={'proc_label': 'label'})
], ignore_index=True).sort_values(['label', 'event_ts']).reset_index(drop=True)
full_event_df

,label,event_ts,event,event_loc,casa
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,SF,senado
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara
4,MPV 1000/2020,2020-09-10 12:09:00,Event.DISTRIBUIDO,MESA,camara
...,...,...,...,...,...
267593,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara
267594,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara
267595,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara
267596,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara


In [25]:
full_event_df = full_event_df[full_event_df[['label', 'event', 'casa']].ne(
    full_event_df[['label', 'event', 'casa']].shift()
).any(axis=1)].reset_index(drop=True)
full_event_df

,label,event_ts,event,event_loc,casa
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,SF,senado
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara
4,MPV 1000/2020,2020-09-10 12:09:00,Event.DISTRIBUIDO,MESA,camara
...,...,...,...,...,...
114703,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara
114704,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara
114705,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara
114706,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara


In [26]:
full_event_df = full_event_df.join(
    house_props_df.set_index('prop_label')['prop_tag'],
    on="label"
).join(
    senate_procs_df.set_index('identificacao')[['tag']].rename(columns={'tag': 'proc_tag'}),
    on="label",
)

In [27]:
full_event_df.to_pickle(ACCESS_DIR / 'full_event_df.pkl')

In [28]:
full_event_df


,label,event_ts,event,event_loc,casa,prop_tag,proc_tag
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,SF,senado,CP:2262062,SP:7979012
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara,CP:2262062,SP:7979012
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara,CP:2262062,SP:7979012
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara,CP:2262062,SP:7979012
4,MPV 1000/2020,2020-09-10 12:09:00,Event.DISTRIBUIDO,MESA,camara,CP:2262062,SP:7979012
...,...,...,...,...,...,...,...
114703,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara,CP:2354532,NaN
114704,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara,CP:2250966,NaN
114705,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara,CP:2284649,NaN
114706,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara,CP:2322707,NaN
